In [40]:
import numpy as np
import pandas as df
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix

In [41]:
filedata = df.read_csv('Churn_Modelling.csv')
filedata.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
label_encoder = LabelEncoder()
filedata['Gender'] = label_encoder.fit_transform(filedata['Gender'])
filedata = df.get_dummies(filedata, columns=['Geography'])
filedata

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,1,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,1,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,0,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,1,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [43]:
features = filedata[['Geography_France', 'Geography_Spain', 'Geography_Germany', 'CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
target = filedata['Exited']
X_train_custom, X_test_custom, y_train_custom, y_test_custom = train_test_split(features, target, test_size=0.2)

sc = StandardScaler()
X_train_custom = sc.fit_transform(X_train_custom)
X_test_custom = sc.transform(X_test_custom)


In [44]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


custom_model = Sequential()

custom_model.add(Dense(units=6, activation='relu', input_dim=12))

custom_model.add(Dense(units=6, activation='relu'))

custom_model.add(Dense(units=1, activation='sigmoid'))

custom_model.compile(loss='mean_squared_error', optimizer='adam')

custom_model.fit(X_train_custom, y_train_custom, batch_size=10, epochs=100)


Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1617
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1380
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1341
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1309
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1281
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1259
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 0.1241
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1226
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1211
Epoch 10/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1198
Epoch 11/100
800/800 [==============================] - 1s 2ms/step - loss: 0.1183
Epoch 12/100
800/800 [==============================] - 2s 2ms/step - loss: 0.1168
Epoch 13/100


In [45]:
new_customer_data = np.array([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])

scaled_new_customer_data = sc.transform(new_customer_data)

pred_prob = custom_model.predict(scaled_new_customer_data)

is_customer_leaving = (pred_prob > 0.5)

if is_customer_leaving:
    print("The new customer is predicted to leave.")
else:
    print("The new customer is predicted to stay.")


1/1 [==============================] - 0s 89ms/step
The new customer is predicted to stay.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [46]:
predictions = custom_model.predict(X_test_custom)
binary_predictions = (predictions > 0.5)

confusion_matrix_custom = confusion_matrix(y_test_custom, binary_predictions)

print("Custom Model Confusion Matrix:\n", confusion_matrix_custom)


63/63 [==============================] - 0s 2ms/step
Custom Model Confusion Matrix:
 [[1527   74]
 [ 221  178]]


**QUESTION 2 CNN**

In [47]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing import image


In [ ]:
from zipfile import ZipFile
file_name = ('dataset.zip')
with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [ ]:
train_data_augmentation = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_data_preprocessing = ImageDataGenerator(rescale=1./255)
training_data = train_data_augmentation.flow_from_directory(
    'your_training_directory_path',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_data = test_data_preprocessing.flow_from_directory(
    'your_test_directory_path',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)




In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

custom_cnn_model = Sequential()

custom_cnn_model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
custom_cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
custom_cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
custom_cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
custom_cnn_model.add(Flatten())
custom_cnn_model.add(Dense(units=128, activation='relu'))
custom_cnn_model.add(Dense(units=1, activation='sigmoid'))

custom_cnn_model.compile(loss='mean_squared_error', optimizer='adam')

custom_cnn_model.fit(training_set, epochs=25, validation_data=test_set)



In [ ]:
image_path = 'dataset/single_prediction/cat_or_dog_1.jpg'
image_size = (64, 64)



loaded_image = image.load_img(image_path, target_size=image_size)
image_array = image.img_to_array(loaded_image)
image_array = np.expand_dims(image_array, axis=0)



custom_model = tf.keras.models.load_model('your_custom_model.h5')
prediction_result = custom_model.predict(image_array)


class_labels = {0: 'cat', 1: 'dog'}
predicted_class = class_labels[0] if prediction_result[0][0] < 0.5 else class_labels[1]

print("Predicted class: " + predicted_class)
